<a href="https://colab.research.google.com/github/wessware/Boosting_Algorithms/blob/main/model/xgb_ordinal_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../datasets/sales_data.csv')

In [3]:
df

,OrderNumber,OrderDate,WarehouseCode,Sales Channel,StoreID,SalesTeamID,CustomerID,ProductID,Order_Quantity,Discount_Applied,Unit_Price,Unit_Cost
0,SO - 000101,5/31/2018,WARE-UHY1004,1,259,6,15,12,5,0.075,1963.1,1001.18
1,SO - 000102,5/31/2018,WARE-NMK1003,2,196,14,20,27,3,0.075,3939.6,3348.66
2,SO - 000103,5/31/2018,WARE-UHY1004,3,213,21,16,16,1,0.050,1775.5,781.22
3,SO - 000104,5/31/2018,WARE-NMK1003,4,107,28,48,23,8,0.075,2324.9,1464.69
4,SO - 000105,5/31/2018,WARE-NMK1003,3,111,22,49,26,8,0.100,1822.4,1476.14
...,...,...,...,...,...,...,...,...,...,...,...,...
7986,SO - 0008087,12/30/2020,WARE-MKL1006,1,339,9,41,29,1,0.075,234.5,121.94
7987,SO - 0008088,12/30/2020,WARE-NMK1003,2,202,14,29,3,6,0.050,3202.6,1921.56
7988,SO - 0008089,12/30/2020,WARE-UHY1004,2,241,14,32,35,5,0.200,3825.7,2792.76
7989,SO - 0008090,12/30/2020,WARE-NMK1003,2,112,20,42,36,8,0.100,1072.0,804.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OrderNumber       7991 non-null   object 
 1   OrderDate         7991 non-null   object 
 2   WarehouseCode     7991 non-null   object 
 3   Sales Channel     7991 non-null   int64  
 4   StoreID           7991 non-null   int64  
 5   SalesTeamID       7991 non-null   int64  
 6   CustomerID        7991 non-null   int64  
 7   ProductID         7991 non-null   int64  
 8   Order_Quantity    7991 non-null   int64  
 9   Discount_Applied  7991 non-null   float64
 10  Unit_Price        7991 non-null   float64
 11  Unit_Cost         7991 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 749.3+ KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Sales Channel,7991.0,1.982856,1.015687,1.00,1.00,2.000,3.00,4.00
StoreID,7991.0,183.850081,105.903946,1.00,91.00,183.000,276.00,367.00
SalesTeamID,7991.0,14.384307,7.986086,1.00,8.00,14.000,21.00,28.00
CustomerID,7991.0,25.457014,14.414883,1.00,13.00,25.000,38.00,50.00
ProductID,7991.0,23.771743,13.526545,1.00,12.00,24.000,36.00,47.00
Order_Quantity,7991.0,4.525341,2.312631,1.00,3.00,5.000,7.00,8.00
Discount_Applied,7991.0,0.114394,0.085570,0.05,0.05,0.075,0.15,0.40
Unit_Price,7991.0,2284.536504,1673.096364,167.50,1031.80,1849.200,3611.30,6566.00
Unit_Cost,7991.0,1431.911513,1112.413063,68.68,606.12,1080.580,2040.25,5498.56


In [6]:
df.isnull().values.any()

False

In [7]:
df.columns

Index(['OrderNumber', 'OrderDate', 'WarehouseCode', 'Sales Channel', 'StoreID',
       'SalesTeamID', 'CustomerID', 'ProductID', 'Order_Quantity',
       'Discount_Applied', 'Unit_Price', 'Unit_Cost'],
      dtype='object')

In [17]:
train_cols = ['Sales Channel', 'StoreID',
       'SalesTeamID', 'CustomerID', 'ProductID',
       'Discount_Applied', 'Unit_Price', 'Unit_Cost']
#'SalesTeamID', 'CustomerID', 'ProductID'
train_cols_few = ['Sales Channel', 'StoreID',
       'Discount_Applied', 'Unit_Price', 'Unit_Cost']

x = df[train_cols_few]
y = df['Order_Quantity']

In [18]:
x.head()

,Sales Channel,StoreID,Discount_Applied,Unit_Price,Unit_Cost
0,1,259,0.075,1963.1,1001.18
1,2,196,0.075,3939.6,3348.66
2,3,213,0.050,1775.5,781.22
3,4,107,0.075,2324.9,1464.69
4,3,111,0.100,1822.4,1476.14


In [19]:
y.head()

0    5
1    3
2    1
3    8
4    8
Name: Order_Quantity, dtype: int64

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)


In [21]:
xgb_mdl = xgb.XGBRegressor()
    
    #colsample_bytree=0.8,
   
    #scale_pos_weight=1,
    #max_depth=3, 
    #n_estimators=400,
    #objective='binary:logistic',
    #booster='gbtree', 
    #n_jobs=-1,
    #random_state=0,
    #num_class=2,
    #learning_rate=0.1,
    #use_label_encoder=False

In [22]:
xgb_mdl.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
predicted = xgb_mdl.predict(x_test)

In [24]:
rmse = np.sqrt(mse(y_test, predicted))

In [25]:
rmse

2.4493963645573262